# iris center - deepeye

# import

In [1]:
import keras
from keras import backend as K

def load_modules():
    import sys
    import numpy as np
    import warnings
    
    sys.path.append("../../../")
    warnings.filterwarnings("ignore")
    np.set_printoptions(precision=6, suppress=True)
    
    %run ../../../ds/icontour/npz/gen.py
    %run ../../../ac/visualizer/plotter.py
    %run ../../../ai/model/iris/deepeye/iris.py
    %run ../../../ai/libs/keras/callbacks/history.py
    %run ../../../ai/libs/keras/callbacks/stopper.py
    %run ../../../ai/libs/keras/callbacks/tensorboard.py
    %run ../../../ai/libs/keras/callbacks/checkpoint.py
        
load_modules()

Using TensorFlow backend.


# backbone model

In [2]:
INPUT_SHAPE=(64, 64, 3)
im = make_deep_eye_net(input_shape=INPUT_SHAPE, num_filters=16, aspp_deep=2, num_classes=2)
im.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 64, 64, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 64, 64, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 64, 64, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

# callback

In [3]:
chk_path = "/home/chy/archive-model/incubator/iris-contour/deepeye-{epoch:02d}-{val_loss:.7f}.hdf5"

history = TimeHistory()
checkpoint = model_checkpoint(chk_path)
tensorboard = TensorBoard()
stopper = EarlyStopping(monitor='val_loss', 
                        min_delta=0, 
                        patience=5, 
                        verbose=0, 
                        mode='auto', 
                        baseline=None, 
                        restore_best_weights=True)

callbacks = [history, checkpoint, stopper, tensorboard]

In [5]:
EXP_CODE = "deepeye-tf"
NUM_EPOCH = 20
BATCH_SIZE = 64

# model save

In [6]:
model_json = im.to_json()
with open("./iris-{}.json".format(EXP_CODE), "w") as json_file : 
    json_file.write(model_json)

# train

In [ ]:
# npz_path = "/home/chy/archive-data/processed/iris-contour-npz/unity"
# npz_path = "/home/chy/archive-data/processed/iris-contour-npz/unity-partial"
npz_path = "/home/chy/archive-data/processed/iris-contour-npz/vc"
# npz_path = "/home/chy/archive-data/processed/iris-contour-npz/vc-partial"

gen_train = IrisContourGenerator(npz_base_path=npz_path,
                                 out_shape=(64, 64),
                                 batch_size=BATCH_SIZE, 
                                 purpose=Purpose.TRAIN,
                                 is_ellipse_mode=False,
                                 use_softmax_pred=True,
                                 use_aug=True)

gen_valid = IrisContourGenerator(npz_base_path=npz_path, 
                                 out_shape=(64, 64),
                                 batch_size=BATCH_SIZE, 
                                 purpose=Purpose.VALID,
                                 is_ellipse_mode=False,
                                 use_softmax_pred=True,
                                 use_aug=False)

In [ ]:
im.fit_generator(generator=gen_train,
                 validation_data=gen_valid,
                 callbacks=callbacks,
                 epochs=NUM_EPOCH,
                 workers=16, 
                 use_multiprocessing=True,
                 shuffle=True)

# test

In [ ]:
gen_valid = IrisContourGenerator(npz_base_path=npz_path, 
                                  batch_size=BATCH_SIZE, 
                                  purpose=Purpose.TEST,
                                  is_ellipse_mode=False,
                                  use_aug=True)